In [1]:
import gzip
import json
import csv
import pandas as pd
import ast

from pandas import json_normalize

import my_functions
%load_ext autoreload
%autoreload 2

Se realiza la carga del archivo 'australian_users_items.json' en el Dataframe 'df_items'.

In [2]:
with open("Datasets/Datasets Originales/australian_users_items.json", 'r', encoding='utf-8') as data:
    data = data.readlines()
    
data = [eval(line) for line in data]

df_items = pd.DataFrame(data)

df_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


Se guarda una versión en crudo en el archivo 'user_items_raw.csv'

In [3]:
df_items.to_csv('Datasets/Datasets ETL/user_items_raw.csv', index=False, encoding='utf-8')

In [4]:
df_items = pd.read_csv('Datasets/Datasets ETL/user_items_raw.csv')

Se comienza la exploración del dataframe.

In [5]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


Campo 'items'

En este campo se encuentran una lista con items (juegos) que posee cada usuario, cada item contenido en un diccionario junto a otra información.

Se procederá a desanidar este campo de forma tal que contaremos con una fila por cada item que posea un usuario.

In [6]:
# Desanidar la columna 'items'
df_items = pd.json_normalize(data, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
df_items.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


Habiendo desanidado la columna 'items' se generaron tantas filas para cada 'user_id' como items posea. Siendo que existen usuarios que poseen muchos juegos, los registros se multiplicaron considerablemente.

In [7]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


Se explora en busca de duplicados.

In [8]:
# Contar filas duplicadas
total_duplicadas = df_items.duplicated().sum()

# Mostrar el total de filas duplicadas
print(f"Total de filas duplicadas: {total_duplicadas}")

Total de filas duplicadas: 59104


Se eliminan más de 59 mil registros duplicados.

In [9]:
df_items = df_items.drop_duplicates(keep='first')

In [10]:
my_functions.describe_df(df_items)

Cantidad Registros:  5094105
Cantidad Campos:  8


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,item_id,object,0,0.0,5094105,100.0,10978,0.22
1,item_name,object,0,0.0,5094105,100.0,10947,0.21
2,playtime_forever,int64,0,0.0,5094105,100.0,48861,0.96
3,playtime_2weeks,int64,0,0.0,5094105,100.0,4450,0.09
4,steam_id,object,0,0.0,5094105,100.0,70912,1.39
5,items_count,object,0,0.0,5094105,100.0,924,0.02
6,user_id,object,0,0.0,5094105,100.0,70912,1.39
7,user_url,object,0,0.0,5094105,100.0,70912,1.39


Tras una primera exploración se determina que los campos 'items_count', 'user_url' y 'playtime_2weeks' no son necesarios, por lo que se los descarta.

In [11]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['items_count', 'user_url', 'playtime_2weeks']

# Realizar el drop de las columnas
df_items = df_items.drop(columnas_a_eliminar, axis=1)


In [12]:
# Mostrar el DataFrame actualizado
df_items.head(3)

,item_id,item_name,playtime_forever,steam_id,user_id
0,10,Counter-Strike,6,76561197970982479,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479,76561197970982479
2,30,Day of Defeat,7,76561197970982479,76561197970982479


Como se realizó en los archivos anteriores, se modifica el nombre de la columna 'item_id' por 'game_id', siendo este el campo compartido por los 3 datasets originales con los que se cuenta.

También se modifica el nombre del campo 'item_name' por 'app_name', correspondiéndose el mismo con el campo 'app_name' de el dataset trabajado en '1.1 ETL games'.

In [13]:
# Cambiar el nombre de la columna 'item_id' a 'game_id'
df_items.rename(columns={'item_id': 'game_id'}, inplace=True)

# Cambiar el nombre de la columna 
df_items.rename(columns={'item_name': 'app_name'}, inplace=True)

Se realiza una comprobación de registros duplicados y al hallar algunos se procede a eliminarlos. Al observar que la única diferencia entre los registros es que algunos tienen un 'playtime_forever' mayor (para el mismo usuario y mismo juego) se eliminan los primeros registros, quedándose aquellos con mayor tiempo de juego.

In [14]:
# Filtrar filas duplicadas
duplicados = df_items[df_items.duplicated(subset=['game_id', 'user_id'], keep=False)]

# Ordenar por game_id y user_id
duplicados = duplicados.sort_values(by=['game_id', 'user_id'])

# Cantidad de filas duplicadas
print('Total filas duplicadas: ', len(duplicados))

# Mostrar resultado
duplicados


Total filas duplicadas:  45


,game_id,app_name,playtime_forever,steam_id,user_id
267027,10190,Call of Duty: Modern Warfare 2 - Multiplayer,111,76561198011268913,everybodylovesrayman
2900346,10190,Call of Duty: Modern Warfare 2 - Multiplayer,111,76561198011268913,everybodylovesrayman
684815,221640,Super Hexagon,1000,76561198018571813,bwolf7803
2132856,221640,Super Hexagon,1001,76561198018571813,bwolf7803
1437604,232090,Killing Floor 2,2231,76561198064484479,76561198064484479
3236765,232090,Killing Floor 2,2231,76561198064484479,76561198064484479
597393,246900,Viscera Cleanup Detail,231,76561198071394519,MIMMATA
2001290,246900,Viscera Cleanup Detail,231,76561198071394519,MIMMATA
597378,251570,7 Days to Die,1906,76561198071394519,MIMMATA
2001275,251570,7 Days to Die,1906,76561198071394519,MIMMATA


In [15]:
# Eliminar duplicados y quedarse con el último registro de cada duplicado
df_items = df_items.drop_duplicates(subset=['game_id', 'user_id'], keep='last')

Se verifican modificaciones.

In [16]:
# Filtrar filas duplicadas
duplicados = df_items[df_items.duplicated(subset=['game_id', 'user_id'], keep=False)]

# Ordenar por game_id y user_id
duplicados = duplicados.sort_values(by=['game_id', 'user_id'])

# Cantidad de filas duplicadas
print('Total filas duplicadas: ', len(duplicados))

# Mostrar resultado
duplicados

Total filas duplicadas:  0


,game_id,app_name,playtime_forever,steam_id,user_id


Se determina eliminar el campo 'steam_id', considerando que el 'user_id' será más útil dado que es un campo compartido con el dataset de reseñas.

In [17]:
# Eliminar 'steam_id'
df_items = df_items.drop('steam_id', axis=1)

Se explora una vez más la estructura de los datos. 
Se determina que los mismos se encuentran en condiciones de ser utilizados para los objetivos propuestos y se guardan en un nuevo archivo, 'user_items_v1.csv'.

In [18]:
my_functions.describe_df(df_items)


Cantidad Registros:  5094082
Cantidad Campos:  4


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,game_id,object,0,0.0,5094082,100.0,10978,0.22
1,app_name,object,0,0.0,5094082,100.0,10947,0.21
2,playtime_forever,int64,0,0.0,5094082,100.0,48861,0.96
3,user_id,object,0,0.0,5094082,100.0,70912,1.39


Se guarda el dataset con los datos limpios.

In [19]:
df_items.to_csv('Datasets/Datasets ETL/user_items_v1.csv', index=False, encoding='utf-8')